# 1D Monitoring Element Exploration

This notebooks provides some examples on how to access and visualize 1D monitoring elements using `cmsdials` and the visualization tools provided by `dqmexplore`.

## Imports

In [ ]:
# Setting up DIALS object and importing dqmexplore
import sys
sys.path.append("../src/")
import dqmexplore as dqme
dials = dqme.utils.setupdials.setup_dials_object_deviceauth()

In [ ]:
# Importing DIALS objects
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

# Importing other useful packages
import numpy as np
import pandas as pd

In [ ]:
# The following function can be used to search for the exact name of the monitoring element you wish to analyze
dqme.utils.datautils.print_availMEs(
    dials, 
    contains="PixelPhase1/Tracks/PXBarrel/charge_PXLayer", 
    dims=1
)

## 1D Monitoring Elements

In [ ]:
# Obtaininng ME data from DIALS
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    max_pages=200
).to_pandas()

medata = dqme.medata.MEData(data1D)

print("Empty LSs for each ME:\n")
for me in medata.getMENames():
    print("{}:".format(me), medata.getEmptyLSs(me))

In [ ]:
# Getting trigger rate and plotting it
trig_rate = dqme.oms.get_rate(dials, runnb)
trig_rate_plot = dqme.oms.plot_rate(
    trig_rate, 
    fig_title="Trigger Rate ({})".format(runnb), 
    norm=False, 
    show=True
)

In [ ]:
# OPTIONAL: Getting reference data 
refrun = 379765

refdata1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = refrun,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    max_pages=200
).to_pandas()

refdata = dqme.medata.MEData(refdata1D)

print("Empty LSs for each ME:\n")
for me in refdata.getMENames():
    print("{}:".format(me), refdata.getEmptyLSs(me))

In [ ]:
trig_rate_ref = dqme.oms.get_rate(dials, refrun)
trig_rate_ref_plot = dqme.oms.plot_rate(
    trig_rate_ref, 
    fig_title="Reference Trigger Rate ({})".format(runnb), 
    norm=False, 
    show=True
)

In [ ]:
# Defining plot features and plotting
ax_labels = [
    dict(
        x="Charge (e)", 
        y="Count"
    )
] * 4

fig_title = f"Pixel Barrel Charge Normalized (Run {runnb})"

# Plotting
fig = dqme.interplt.plot1DMEs(
    medata, 
    fig_title=fig_title,
    ax_labels=ax_labels, 
    width=1000,
    height=1000,
    vspace=0.1,
    hspace=0.1,
    # trigger_rates=trig_rate, 
    norm=True,
    ref_data=refdata,
    show=True
)

The figure constructed above can be further modified using `plotly`.

In [ ]:
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add modifications to fig here.


And you can also save the figure by running the folling cell.

In [ ]:
# You can set show=False and export the figure to an HTML file by also running the code below
from plotly.offline import plot
plot(fig, filename=f"./fig.html")

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken. 


These can also be exported to an HTML file similarly to the example shown previously.

In [ ]:
# Getting current run data
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_." 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()

In [ ]:
# Getting trigger rate
trig_rate = dqme.oms.get_rate(dials, runnb)

In [ ]:
# Defining plot features and plotting
ax_labels = [
    dict(x = "Charge(e)", y = "LS")
] * 4

fig_title = f"Pixel Barrel Charge Heatmaps Normalized by Trigger Rate (Run {runnb})"

fig = dqme.staticplt.plotheatmaps1D(
    medata,
    fig_title=fig_title,
    ax_labels=ax_labels,
    # trigger_rates = trig_rate,
    # norm=True,
    show=True,
)